<a href="https://colab.research.google.com/github/ayushpattnaik/SOB_Ayush/blob/main/SummerOfBitcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Uploading the mempool csv file

In [ ]:
import csv


In [ ]:
from google.colab import files
uploaded=files.upload()

Saving mempool.csv to mempool.csv


### Reading mempool.csv to dataframe df

In [ ]:
import pandas as pd
import io
df=pd.read_csv(io.BytesIO(uploaded['mempool.csv']))

In [ ]:
df

,tx_id,fee,weight,parents
0,2e3da8fbc1eaca8ed9b7c2db9e6545d8ccac3c67deadee...,452,1620,NaN
1,79c51c9d4124c5cbb37a85263748dcf44e182dff83561f...,682,1136,6eb38fad135e38a93cb47a15a5f953cbc0563fd84bf1ab...
2,b0ef627c8dc2a706475d33d7712209ec779f7a8302aaea...,226,900,NaN
3,8c25f9be93990b96e8bc363778d6debee6867c7d73cefa...,258,1032,NaN
4,edc82751537be6afcb21c0229d8d20916d8e742a363ab2...,452,1616,NaN
...,...,...,...,...
5209,e0b5d2fe598fccdd716df743323346da227dfc9d2a6c3e...,850,568,NaN
5210,e33973618a9774aeb85548368efeaddb206ef206717e72...,1496,1488,NaN
5211,88a8c7118ac82199523eb10670f49c8b7f5ac82d404907...,1121,900,33c7c7bcb1cc07743fc3240cac6af31e4d4a8f535cae71...
5212,4c5b18c0870665958fc2a377e425b70f8d01f03395af72...,862,576,NaN


In [ ]:
df['weight'].sum()

10431600

### Sorting the dataframe according to fee for maximizing the fees

In [ ]:
df2=df.sort_values(by=['fee'],ascending=False)

In [ ]:
df2

,tx_id,fee,weight,parents
2660,59f0495cf66d1864359dda816eb7189b9d9a3a9cd9dc50...,201700,1348,NaN
4693,3bfc4c22fc7aaded4b02c6a6d67b4a7bad297377e46e4c...,194166,3852,NaN
4877,87784075804f10dad1f815de867dde2875e73a13da798c...,107775,1660,NaN
4908,0c8ebf9c75f63b7e5ff176e2937f24c694aa6b3bde0e59...,104400,2084,NaN
5095,c3fef085fca34891e6456489d840ab68139b24857eb1f9...,100000,1148,NaN
...,...,...,...,...
816,dbf57df85f4292e2211439f44974f3d99df18fbbe30f05...,114,452,NaN
883,67d2b9c9c9755ecd55d7528f576845eb2f3d982b25c23f...,111,444,NaN
1664,89fa28f476f4f7fea0ffd3d8e687a1bf94e1ed96cb357d...,111,440,NaN
1662,62e84b510b0d3e05725d2e626709cee88b9d5f78956656...,111,440,NaN


### Finding the index where the maximum weight of the block is achieved

In [ ]:
last_index=pd.Index(df2.weight.cumsum()).get_loc(4000000,'nearest')

In [ ]:
last_index

150

In [ ]:
df2.loc[[last_index]]

,tx_id,fee,weight,parents
150,f39b102a3349a6de9b80950f4d731e9bc0b349d6dc8a10...,452,1612,8580ab895e5085eefd7e9e9016d017f9bdf8b64bba425c...


### Loading the dataframe with maximum fee and maximum weight capacity 

In [ ]:
df3=df2.iloc[0:last_index,[0]]

In [ ]:
df3

,tx_id
2660,59f0495cf66d1864359dda816eb7189b9d9a3a9cd9dc50...
4693,3bfc4c22fc7aaded4b02c6a6d67b4a7bad297377e46e4c...
4877,87784075804f10dad1f815de867dde2875e73a13da798c...
4908,0c8ebf9c75f63b7e5ff176e2937f24c694aa6b3bde0e59...
5095,c3fef085fca34891e6456489d840ab68139b24857eb1f9...
...,...
1051,12bfaccecfd4355bde2e4073015624a131d23a33e30160...
917,573bd82b65d312ac1f0ff5e502937200963e87467a954e...
782,82ae3ab0db20ff81110dc62e4d4b943197f44e364ecd2a...
2353,cbacf7645d5e443210bcfb16dee9ea76c60cd66c3f7b3f...


In [ ]:
df3.to_csv('block.txt')

### Alogrithm for sorting the parent_txids

I wrote this algorithm because I am not much familiar with python csv handling.

**The Algorithm:**

For every tx_id (say **curr_index**) in dataframe d3 with valid parent txids: 
- Check if all its parent txids already exist before the **curr_index** tx_id.
  1. If the parent_txids occur after the **curr_index** within the dataframe then move the parent_txids before the **curr_index**.

  2. If the parent_txid doesn't occur within the list, extract the first index from the sorted dataframe d2 after 
the **last_index** which is **last_index+1**
   >  Check if the tx_id of **last_index+1** is a parent txid of **curr_index** tx_id. 
     - If yes, append the **last_index+1**  before the curr_index and pop the **last_index** from the list.
     - If not, find the next index(**say x**) of the parent tx_id occuring after **last_index+1**  and  find the sum(**say sum1**) of the fees of the **curr_index**  and index **x** and the sum(**say sum2**) of the fees of **last_index+1**  and **last_index**. <br>
       ```sum1=fee[x]+fee[curr_index]```<br>
       ```sum2=fee[last_index]+fee[last_index+1]```<br>
      **If sum1>sum2:**<br>
            Append the index(x) before the curr_index and delete the last_index from the list.
      **Else If sum1<sum2:**
            Delete the curr_index and append the last_index+1 to the list.

  3. Repeat the process 1 and 2 for every parent_txid of curr_index

        
